In [ ]:
import bq_helper
from bq_helper import BigQueryHelper

stackOverflow = BigQueryHelper(active_project='bigquery-public-data', dataset_name='stackoverflow')

In [ ]:
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")
bq_assistant.list_tables()

In [ ]:
#query = 'SELECT x.id, x.body FROM bigquery-public-data.stackoverflow.posts_questions as x WHERE x.tags LIKE "%java%" OR x.tags LIKE "%android%" AND x.body LIKE "%<code>%"'
#query = 'SELECT COUNT(x) FROM bigquery-public-data.stackoverflow.posts_answers as x WHERE x.body LIKE "%<code>%"'
#response = stackOverflow.query_to_pandas_safe(query, max_gb_scanned=30)
#response

#query = 'SELECT ide, REGEXP_EXTRACT_ALL(body, r\'(<code>[\s\S]*?<\/code>)\') as code FROM (SELECT x.id as ide, x.body as body FROM bigquery-public-data.stackoverflow.posts_questions as x WHERE x.tags LIKE "%java%" OR x.tags LIKE "%android%" AND x.body LIKE "%<code>%")'
#response2 = stackOverflow.query_to_pandas_safe(query, max_gb_scanned=30)
#response2

#query3 = 'SELECT answer_id, LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", "")) FROM (SELECT answer_id, REGEXP_EXTRACT_ALL(answer_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet FROM(SELECT answer.id as answer_id,answer.body as answer_body FROM bigquery-public-data.stackoverflow.posts_answers as answer JOIN bigquery-public-data.stackoverflow.posts_questions as question ON answer.parent_id = question.id WHERE question.tags LIKE "java" OR question.tags LIKE "%android%" AND answer.body LIKE "%<code>%")), UNNEST(code_snippet) unnested_code_snippets'
query3 = 'SELECT answer_id, LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", "")) FROM (SELECT answer_id, REGEXP_EXTRACT_ALL(answer_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet FROM(SELECT answer.id as answer_id,answer.body as answer_body FROM bigquery-public-data.stackoverflow.posts_answers as answer JOIN bigquery-public-data.stackoverflow.posts_questions as question ON answer.parent_id = question.id WHERE question.tags LIKE "java" AND answer.body LIKE "%<code>%")), UNNEST(code_snippet) unnested_code_snippets'

response3 = stackOverflow.query_to_pandas_safe(query3, max_gb_scanned=30)
response3

In [ ]:
new_lines_in_snippets_df = response3

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#normalized_mean_df=(new_lines_in_snippets_df-new_lines_in_snippets_df.mean())/new_lines_in_snippets_df.std()
plt.hist(new_lines_in_snippets_df['f0_'], bins=range(3, 20))

In [ ]:
total_snippets = len(new_lines_in_snippets_df.index)
print('Total snippets: %d' % total_snippets)

Filter out all code snippets tahat are less than four lines loong using the elbow method. First try to get all in one dataset. If that fails, try to load batch.

In [ ]:
#query4 = 'SELECT answer_id, unnested_code_snippets FROM (SELECT answer_id, REGEXP_EXTRACT_ALL(answer_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet FROM(SELECT answer.id as answer_id,answer.body as answer_body FROM bigquery-public-data.stackoverflow.posts_answers as answer JOIN bigquery-public-data.stackoverflow.posts_questions as question ON answer.parent_id = question.id WHERE question.tags LIKE "java" OR question.tags LIKE "%android%" AND answer.body LIKE "%<code>%")), UNNEST(code_snippet) unnested_code_snippets WHERE (LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", ""))) >= 4
#query4 = 'SELECT answer_id, unnested_code_snippets FROM (SELECT answer_id, REGEXP_EXTRACT_ALL(answer_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet FROM(SELECT answer.id as answer_id,answer.body as answer_body FROM bigquery-public-data.stackoverflow.posts_answers as answer JOIN bigquery-public-data.stackoverflow.posts_questions as question ON answer.parent_id = question.id WHERE question.tags = "java" AND answer.body LIKE "%<code>%")), UNNEST(code_snippet) unnested_code_snippets WHERE (LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", ""))) >= 3'

query4 = '''
SELECT answer_id as id, unnested_code_snippets FROM (SELECT answer_id, REGEXP_EXTRACT_ALL(answer_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet FROM(SELECT answer.id as answer_id,answer.body as answer_body FROM bigquery-public-data.stackoverflow.posts_answers as answer JOIN bigquery-public-data.stackoverflow.posts_questions as question ON answer.parent_id = question.id WHERE question.tags = "java" AND answer.body LIKE "%<code>%")), UNNEST(code_snippet) unnested_code_snippets WHERE (LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", ""))) >= 4

UNION ALL

(SELECT question_id as id, unnested_code_snippets FROM 
    (SELECT question_id, REGEXP_EXTRACT_ALL(question_body, r"(<code>[\s\S]*?<\/code>)") as code_snippet 
    FROM
    (SELECT question.id as question_id, question.body as question_body 
    FROM bigquery-public-data.stackoverflow.posts_questions as question
    WHERE question.tags = "java" AND question.body LIKE "%<code>%")), 
    UNNEST(code_snippet) unnested_code_snippets 
    WHERE (LENGTH(unnested_code_snippets) - LENGTH(REGEXP_REPLACE(unnested_code_snippets, "\\n", ""))) >= 4)
'''

code_snippets = stackOverflow.query_to_pandas_safe(query4, max_gb_scanned=60)

In [ ]:
code_snippets_num = len(code_snippets.index)
print('Number of snippets: %d\nNumber of 4+ snippets: %d\nReduction: %f' % (total_snippets, code_snippets_num, code_snippets_num / total_snippets))

In [ ]:
code_snippets

In [ ]:
code_snippets['unnested_code_snippets'].str.len().plot.hist(bins=500, legend=True)

In [ ]:
code_snippets['unnested_code_snippets'].str.len().describe()

In [ ]:
i = 0
for s in code_snippets['unnested_code_snippets']:
    if s.count('\n') == 15 and i < 10:
        print(s)
        i += 1

In [ ]:
code_snippets.to_csv('java_questions_and_answers_4+_snippets.csv', index=False)

In [ ]:
java_tokens = '''
abstract
assert
boolean
break
byte
case
catch
char
class
continue
default
do
double
else
enum
extends
final
finally
float
for
if
implements
import
instanceof
int
interface
long
native
new
package
private
protected
public
return
short
static
strictfp
super
switch
synchronized
this
throw
throws
transient
try
void
volatile
while
'''
java_tokens = java_tokens.split('\n')[1:-1]
java_tokens

## Data obtained

In [ ]:
import pandas as pd

code_snippets = pd.read_csv('./java_questions_and_answers_4+_snippets.csv')

In [ ]:
code_snippets['unnested_code_snippets'].str.len().plot.hist(bins=100)

In [ ]:
print(code_snippets.shape)

In [ ]:
'|'.join(java_tokens)

In [ ]:
new_df = code_snippets[code_snippets['unnested_code_snippets'].str.contains('|'.join(java_tokens))]

In [ ]:
new_df.shape[0] / code_snippets.shape[0]

In [ ]:
code_snippets = new_df

In [ ]:
from html.parser import HTMLParser
import html

start_tag = '<code>'
end_tag = '</code>'

parser = HTMLParser()
raw_snippet = code_snippets.at[0, 'unnested_code_snippets'][len(start_tag):-len(end_tag)]
print(raw_snippet)

In [ ]:
decoded = html.unescape(raw_snippet)
print(decoded)

## Create blob

snip1, snip2, snip3, snip4, snip5, snip6....

'\n'.join(html.unescape(snip) for snip in all_snippets)

snip1Xsnip2Xsnip3Xsnip4X....

len(snip1)
len(snip1) + len(snip2) + 1
label(n) = label(n - 1) + label(n) + 1

dividing_newlines = dict()
len(snip1)
len(snip1) + len(snip2)

In [ ]:
snippets_list = code_snippets['unnested_code_snippets'].to_numpy()

In [ ]:
def force_ascii(string):
    encoded_string = string.encode('ascii', 'ignore')
    return encoded_string.decode()

In [ ]:
clean_snippets = [force_ascii(html.unescape(raw_snippet[len(start_tag):-len(end_tag)]).strip('\n \t\r')) for raw_snippet in snippets_list]

In [ ]:
for i in range(100, 500):
    print(clean_snippets[i])

In [ ]:
giant_block = '\n'.join(clean_snippets)

In [ ]:
code_snippets.shape[0] / len(giant_block)

In [ ]:
segment_new_lines = [len(clean_snippets[0])]
for i in range(1, len(clean_snippets)):
    segment_new_lines.append(segment_new_lines[i - 1] +  len(clean_snippets[i]) + 1)

In [ ]:
giant_block[segment_new_lines[123]]

In [ ]:
!pip install algorithms

In [ ]:
import pickle
from tqdm import tqdm
import gc
import os
import numpy as np

def datapoints(code_block, window_size, dividing_newlines):
    i = 0
    label_container = []
    window_container = []
    total = 0 # samples
    
    for window_start in tqdm(range(0, len(code_block), window_size)):
        labels = []
        window = []
        
        for ch_index in range(window_start, min(window_start+window_size, len(code_block))):
            if dividing_newlines[i] != ch_index:
                labels.append(0)
            else:
                labels.append(1)
                i += 1
            window.append(ord(code_block[ch_index]))

        if len(window) == window_size:
          label_container.append(np.array(labels, dtype='int'))
          window_container.append(np.array(window, dtype='float64'))
          total += 1

        #window_np = [ord(ch) for ch in code_block[window_start:window_start+window_size]]
        #labels_np = labels
        
        #yield (window_np, labels_np)
        #yield labels
    
    return window_container, label_container

windows, labels = datapoints(giant_block, 100, segment_new_lines)
windows = np.array(windows, dtype='float64')
np.save('java_questions_and_answers_4+_windows.npy', windows)

In [ ]:
del windows

In [ ]:
labels = np.array(labels, dtype='int')
np.save('java_labels.npy', labels)

In [ ]:
from algorithms.search import binary_search
import numpy as np
import h5py
import pickle
from tqdm import tqdm
import gc
import os

CHUNK_CAPACITY = 2**17

''' return (window, labels) '''
def datapoints(code_block, window_size, dividing_newlines):
    i = 0
    label_container = []
    window_container = []
    chunk = 0
    total = 0
    
    if not os.path.exists('tmp'):
        os.mkdir('tmp')
    
    for window_start in tqdm(range(0, len(code_block), window_size)):
        labels = []
        window = []
        
        for ch_index in range(window_start, min(window_start+window_size, len(code_block))):
            if dividing_newlines[i] != ch_index:
                labels.append(0)
            else:
                labels.append(1)
                i += 1
            window.append(ord(code_block[ch_index]))
        
        label_container.append(np.array(labels))
        window_container.append(np.array(window))
        total += 1
        
        if len(window_container) == CHUNK_CAPACITY:
            chunkd = {'windows': window_container, 'labels': label_container}
            with open('tmp/x_chunk_%d.pickle' % (chunk,), 'wb') as handle:
                pickle.dump(chunkd, handle, protocol=pickle.HIGHEST_PROTOCOL)
            chunk += 1
            label_container = []
            window_container = []
    
    # cleaning
    if len(window_container) > 0:
        chunkd = {'windows': window_container, 'labels': label_container}
        with open('tmp/x_chunk_%d.pickle' % (chunk,), 'wb') as handle:
            pickle.dump(chunkd, handle, protocol=pickle.HIGHEST_PROTOCOL)
        chunk += 1
        del chunkd
        label_container = []
        window_container = []
                
        #window_np = [ord(ch) for ch in code_block[window_start:window_start+window_size]]
        #labels_np = labels
        
        #yield (window_np, labels_np)
        #yield labels
    
    return total, chunk

total, chunks = datapoints(giant_block, 100, segment_new_lines)
#points = 0
#for datapoint in datapoints(giant_block, 100, segment_new_lines):
#    points += 1
#labels, frame = datapoints(giant_block, 100, segment_new_lines)

#window, labels = next(g)
#len(window) == len(labels)

Determine chunk load time

In [ ]:
import time

watch_point = time.time()

with open('tmp/x_chunk_0.pickle', 'rb') as handle:
    b = pickle.load(handle)

print('[' + str(CHUNK_CAPACITY) + '] load time: ' + str(time.time() - watch_point))

In [ ]:
# try without last
total_new = int(total / CHUNK_CAPACITY) * CHUNK_CAPACITY
chunks_new = int(total / CHUNK_CAPACITY)
print(total_new / total)

## Create Train/Validation/Test datasets

......................... giant block .............................
                   train                     |  valid     |  test
                   
train_size = int(len(giant_block) * .8); test_size = int(len(giant_block) * .1); valid_size = int(len(giant_block) * .1)


train_start = 0; train_end = train_size


valid_start = train_end; valid_end = valid_start + valid_size


test_start = valid_end; test_end = test_start + test_size

In [ ]:
from keras.utils import Sequence
import random

class DataGenerator(Sequence):
    def __init__(self, giant_block, index_start, index_end, window_size, dividing_newlines, batch_size):
        self.giant_block = giant_block
        self.index_start = index_start
        self.index_end = index_end
        self.window_size = window_size
        self.dividing_newlines = dividing_newlines
        self.batch_size = batch_size
        
        self._indices = list(range(0, len(self)))
        self.on_epoch_end()
        
        
    def __len__(self):
        samples = (self.index_end - self.index_start) / self.window_size
        return int(np.floor(samples / self.batch_size))
    
    
    def __getitem__(self, index):
        mapped_index = self._indices[index] # make it random
        batch_index_start = self.index_start + mapped_index * self.window_size
        
        labels = []
        
        for ch_index in range(batch_index_start, batch_index_start + self.window_size):
            index = binary_search(self.dividing_newlines, ch_index)
            
            if index is None:
                labels.append(0)
            else:
                labels.append(1)
                
        window = [ord(ch) for ch in self.giant_block[batch_index_start: batch_index_start + self.window_size]]
        
        return np.array(window), np.array(labels)
    
        
    def on_epoch_end(self):
        random.shuffle(self._indices)
    
#ds = DataGenerator(giant_block, 0, int(len(giant_block) * .8), 100, segment_new_lines, 128)

class GDataGenerator(Sequence):
    def __init__(self, chunks_names, batch_size):
        self.total_chunks = len(chunks_names)
        self.total_samples = self.total_chunks * CHUNK_CAPACITY
        self.batch_size = batch_size
        self.chunks_names = chunks_names
        
        self.chunk_indices = list(range(self.total_chunks)) # take care of last
        
        self.loaded_chunks = 0
        self.loaded_chunk = None
        
        self.on_epoch_end()
        
    # FOR NOW PROVIDE FULL
    def __len__(self):
        return int(self.total_samples / self.batch_size)
    
    def __getitem__(self, index):
        if not (index < self.loaded_chunks * CHUNK_CAPACITY):
            with open('tmp/x_chunk_%d.pickle' % (self.chunks_names[self.loaded_chunks]), 'rb') as handle:
                self.loaded_chunk = pickle.load(handle)
                self.loaded_chunks += 1
        
        return self.loaded_chunk['windows'][index % CHUNK_CAPACITY], self.loaded_chunk['labels'][index % CHUNK_CAPACITY]
    
    def on_epoch_end(self):
        self.chunk_indices = list(range(self.total_chunks - 1))
        random.shuffle(self.chunk_indices)
        self.chunk_indices.append(self.total_chunks - 1) # because last is not full

d = GDataGenerator(range(chunks_new), 128)

In [ ]:
from keras.utils import Sequence
import random

class GDataGenerator(Sequence):
    def __init__(self, chunks_names, batch_size):
        self.total_chunks = len(chunks_names)
        self.total_samples = self.total_chunks * CHUNK_CAPACITY
        self.batch_size = batch_size
        self.chunks_names = chunks_names
        
        self.chunk_indices = list(range(self.total_chunks)) # take care of last
        
        self.loaded_chunks = 0
        self.loaded_chunk = None
        
        self.on_epoch_end()
        
    # FOR NOW PROVIDE FULL
    def __len__(self):
        return int(self.total_samples / self.batch_size)
    
    def __getitem__(self, index):
        if not (index < self.loaded_chunks * CHUNK_CAPACITY):
            with open('tmp/x_chunk_%d.pickle' % (self.chunks_names[self.loaded_chunks]), 'rb') as handle:
                self.loaded_chunk = pickle.load(handle)
                self.loaded_chunks += 1

        #X = np.empty((128, 100))

        X = np.array(self.loaded_chunk['windows'][index % CHUNK_CAPACITY: index % CHUNK_CAPACITY + self.batch_size], dtype='float64')
        y = np.array(self.loaded_chunk['labels'][index % CHUNK_CAPACITY: index % CHUNK_CAPACITY + self.batch_size], dtype='float64')
        
        return X, y
    

    def on_epoch_end(self):
        print('shuffle')
        self.chunk_indices = list(range(self.total_chunks - 1))
        random.shuffle(self.chunk_indices)
        self.chunk_indices.append(self.total_chunks - 1) # because last is not full

In [ ]:
# Procentage of true positive
100 * len(segment_new_lines) / len(giant_block)

In [ ]:
train_size = int(len(giant_block) * .8)
test_size = int(len(giant_block) * .1)
valid_size = int(len(giant_block) * .1)

train_start = 0
train_end = train_size

valid_start = train_end
valid_end = valid_start + valid_size

test_start = valid_end
test_end = test_start + test_size


# gen
all_chunks = list(range(chunks_new))

gtrain_lim = int(len(all_chunks) * .8)
gvalid_lim = gtrain_lim + int(len(all_chunks) * .1)

gtrain = all_chunks[:gtrain_lim]
gvalid = all_chunks[gtrain_lim:gvalid_lim]
gtest = all_chunks[gvalid_lim:]
len(gtrain)/len(all_chunks), len(gvalid)/len(all_chunks), len(gtest) / len(all_chunks)

In [ ]:
'''
data_points_x = []
data_points_y = []
i = 0
for dp in datapoints(giant_block, 100, segment_new_lines):
    data_points_x.append(dp[0])
    data_points_y.append(dp[1])
'''
#training_generator = DataGenerator(giant_block, train_start, train_end, 100, segment_new_lines, 128)
#validation_generator = DataGenerator(giant_block, valid_start, valid_end, 100, segment_new_lines, 128)

gtraining_generator = GDataGenerator(gtrain, 128)
gvalidation_generator = GDataGenerator(gvalid, 128)

In [ ]:
from keras import backend as K

def positive_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    return true_positives

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Bidirectional, LSTM, Dense, TimeDistributed, Flatten
from keras.metrics import AUC

n_vocab = 256 
num_lstm_units = 256 
bimodel = Sequential() 
bimodel.add(Embedding(n_vocab,20, input_length =100)) 
bimodel.add(Dropout(.2)) 
bimodel.add(Bidirectional(LSTM(num_lstm_units, return_sequences=True))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(150, activation='relu'))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(75, activation = 'relu'))) 
bimodel.add(Dropout(.2)) 
bimodel.add(TimeDistributed(Dense(1, activation = 'sigmoid'))) 
bimodel.add(Flatten())
bimodel.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['acc', AUC(curve='PR')]) 

> model = bimodel.fit(
>     train_x,
>     train_y,
>     batch_size=128,
>     epochs=20,
>     validation_data=(val_x, val_y))

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'model-{epoch}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    period=4)

In [ ]:
model = bimodel.fit_generator(
    generator=gtraining_generator,
    validation_data=gvalidation_generator,
    use_multiprocessing=True,
    workers=4,
    callbacks=[model_checkpoint_callback],
    epochs=1000)